In [1]:
import os
import random
from math import ceil
from contextlib import redirect_stdout
import tensorflow
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, train_test_split
from IPython.display import display
from pytictoc import TicToc
from utils import PlotLosses, grafica_kfold

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201
from tensorflow.keras.applications.nasnet import NASNetLarge, NASNetMobile

gpus = tensorflow.config.experimental.list_physical_devices('GPU')

tensorflow.config.experimental.set_visible_devices(gpus[1], 'GPU')
tensorflow.compat.v1.disable_eager_execution()

ruta = os.path.abspath(r'./')
directorio_experimento = os.path.join(ruta, f'experimento_transfer')
if not os.path.exists(directorio_experimento):
    os.mkdir(directorio_experimento)
os.chdir(directorio_experimento)

In [2]:
def modelo_base(model):
    if model == "VGG16":
        from tensorflow.keras.applications.vgg16 import preprocess_input
        preprocessing_function = preprocess_input
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))
        return base_model, preprocessing_function
    elif model == "VGG19":
        from tensorflow.keras.applications.vgg19 import preprocess_input
        preprocessing_function = preprocess_input
        base_model = VGG19(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))
        return base_model, preprocessing_function
    elif model == "ResNet50":
        from tensorflow.keras.applications.resnet50 import preprocess_input
        preprocessing_function = preprocess_input
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))
        return base_model, preprocessing_function
    elif model == "InceptionV3":
        from tensorflow.keras.applications.inception_v3 import preprocess_input
        preprocessing_function = preprocess_input
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))
        return base_model, preprocessing_function
    elif model == "Xception":
        from tensorflow.keras.applications.xception import preprocess_input
        preprocessing_function = preprocess_input
        base_model = Xception(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))
        return base_model, preprocessing_function
    elif model == "InceptionResNetV2":
        from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
        preprocessing_function = preprocess_input
        base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))
        return base_model, preprocessing_function
    elif model == "MobileNet":
        from tensorflow.keras.applications.mobilenet import preprocess_input
        preprocessing_function = preprocess_input
        base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))
        return base_model, preprocessing_function
    elif model == "MobileNetV2":
        from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
        preprocessing_function = preprocess_input
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))
        return base_model, preprocessing_function
    elif model == "DenseNet121":
        from tensorflow.keras.applications.densenet import preprocess_input
        preprocessing_function = preprocess_input
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))
        return base_model, preprocessing_function
    elif model == "DenseNet169":
        from tensorflow.keras.applications.densenet import preprocess_input
        preprocessing_function = preprocess_input
        base_model = DenseNet169(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))
        return base_model, preprocessing_function
    elif model == "DenseNet201":
        from tensorflow.keras.applications.densenet import preprocess_input
        preprocessing_function = preprocess_input
        base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))
        return base_model, preprocessing_function
    elif model == "NASNetLarge":
        from tensorflow.keras.applications.nasnet import preprocess_input
        preprocessing_function = preprocess_input
        base_model = NASNetLarge(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))
        return base_model, preprocessing_function
    elif model == "NASNetMobile":
        from tensorflow.keras.applications.nasnet import preprocess_input
        preprocessing_function = preprocess_input
        base_model = NASNetMobile(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))
        return base_model, preprocessing_function
    else:
        ValueError("The model you requested is not supported in Keras")


def build_finetune_model(base_model, dropout, fc_layers, num_classes, opt):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = tensorflow.keras.layers.Flatten()(x)
    for fc in fc_layers:
        x = tensorflow.keras.layers.Dense(fc, activation='relu')(x) # New FC layer, random init
        x = tensorflow.keras.layers.Dropout(dropout)(x)

    predictions = tensorflow.keras.layers.Dense(num_classes, activation='softmax')(x) # New softmax layer
    
    finetune_model = tensorflow.keras.Model(inputs=base_model.input, outputs=predictions)

    finetune_model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return finetune_model

def train(modelo, train, val):

    print('Construyendo modelo')
    metrics = ['accuracy']
    mo, preprocessing_function = modelo_base(modelo)
    if not os.path.exists(os.path.join(directorio_experimento, modelo)):
        os.mkdir(os.path.join(directorio_experimento, modelo))

    print('Creando generadores')

    train_datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=preprocessing_function,
        horizontal_flip=True,
        vertical_flip=True,
        # rotation_range=25,
        fill_mode='constant'
        
    )
    
    test_datagen =tensorflow.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=preprocessing_function,
    )
    
    train_generator = train_datagen.flow_from_dataframe(
                                          train,
                                          None,
                                          x_col='file',
                                          target_size=(WIDTH, HEIGHT),
                                          y_col=f'Class_cat_7', 
                                          batch_size=BATCH_SIZE, 
                                          seed=SEED,
                                          has_ext=True,class_mode='categorical')

    
    test_generator = test_datagen.flow_from_dataframe(val, 
                                          None, 
                                          x_col='file', 
                                          target_size=(WIDTH, HEIGHT),
                                          y_col=f'Class_cat_7', 
                                          batch_size=BATCH_SIZE,
                                          seed=SEED,
                                          has_ext=True,
                                          class_mode='categorical', 
                                          shuffle=True)
    class_list = list(test_generator.class_indices.keys())
    model = build_finetune_model(mo, DROPOUT, FC_LAYERS, num_classes=len(class_list), opt=OPT)

    plot_losses = PlotLosses(figsize=(10,6))
    callbacks_list = [
                      plot_losses,  
                      tensorflow.keras.callbacks.CSVLogger(os.path.join(os.path.join(directorio_experimento, modelo), 'log.csv'))
    ]
    print('Iniciando entrenamiento')
    history = model.fit(train_generator, 
                        epochs=EPOCHS, 
                        workers=16, 
                        shuffle=True, 
                        callbacks=callbacks_list, 
                        verbose=1, 
                        steps_per_epoch=ceil(len(train.index) / (BATCH_SIZE * 2)),
                        validation_data=test_generator, 
                        validation_steps=ceil(len(val.index) / (BATCH_SIZE * 2))
                                 )


In [ ]:
WIDTH = 224
HEIGHT = 224
EPOCHS = 20
DROPOUT = 0.5
EVALUACIONES = []
BATCH_SIZE = 64
FC_LAYERS = [1024 * 1, 1024  * 1]  # 1024
LR = 1e-5
OPT = tensorflow.keras.optimizers.Adam(lr=LR)
SEED = 111091
np.random.seed(SEED)
EPOCHS = 10 # 60
avg_acc = []
avg_loss = []

datos = pd.read_csv(f'{ruta}\\database\\bd_aumentada\\database_augment.csv')

datos_random = datos.sample(frac=1).reset_index(drop=True)

HYP = dict(batch_size=BATCH_SIZE, 
           width=WIDTH, 
           height=HEIGHT, 
           learning_rate=LR, 
           optimizer=str(OPT), 
           seed=SEED, 
           epochs=EPOCHS, 
           dropout=str(DROPOUT), 
           fc_layers=str(FC_LAYERS),
)

HYP = {'Valor': [BATCH_SIZE, 
                 WIDTH, 
                 HEIGHT, 
                 LR, 
                 OPT, 
                 SEED, 
                 EPOCHS, 
                 DROPOUT, 
                 FC_LAYERS]}
df_hyp = pd.DataFrame(HYP, 
                      index=['BATCH_SIZE', 
                             'WIDTH', 
                             'HEIGHT', 
                             'LR', 
                             'OPT', 
                             'SEED', 
                             'EPOCHS', 
                             'DROPOUT', 
                             'FC_LAYERS']
                     )
display(df_hyp)
df_hyp.to_csv('hiperparametros.csv')


preprocessing_function = None
base_model = None

modelos = [
            'VGG16', 
            'VGG19', 
            'Xception', 
             'ResNet50',
            'InceptionV3', 
            'InceptionResNetV2', 
            'MobileNet', 
            'MobileNetV2', 
            'DenseNet201', 
            'NASNetMobile'
            'NASNetLarge']
msk = np.random.rand(len(datos_random)) < 0.8
for ele in modelos:
    print(f'Analizando: {ele}')
    train(ele, datos_random[msk], datos_random[~msk])